# Templatized notebook for running CB-Geo MPM TAPIS job

## Install DesignSafe API (dapi)

In [ ]:
!pip install --user --ignore-installed dapi==0.4.2 --quiet

In [ ]:
import sys, os

print(
    "Old NumPy 1.24 installed!"
    if sys.version_info < (3, 10)
    and os.system("pip install --user --force-reinstall numpy~=1.24.0 --quiet") == 0
    else "Skipped (Python 3.10+)"
    if sys.version_info >= (3, 10)
    else "Install failed!"
)

### ⚠️ Please restart the kernel by choosing Kernel >> Restart kernel ⚠️

In [ ]:
import numpy as np

print(np.__version__)

In [1]:
# Import required modules
from dapi import DSClient
import json

/Users/krishna/dev/DesignSafe/dapi/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialize DesignSafe client
ds = DSClient()

Authentication successful.
DatabaseAccessor initialized. Connections will be created on first access.


### Get MPM Benchmark directory

In [ ]:
![ ! -d ~/MyData/mpm-benchmarks-develop ] && wget https://github.com/cb-geo/mpm-benchmarks/archive/refs/heads/develop.zip -P ~/MyData/ && cd ~/MyData/ && unzip develop.zip && rm develop.zip

### MPM Job configuration

In [3]:
# Job configuration parameters
ds_path: str = "/MyData/mpm-benchmarks/2d/uniaxial_stress/"  # Path to input files
input_filename: str = "mpm.json"  # Main input script filename
max_job_minutes: int = 10  # Maximum runtime in minutes
tacc_allocation: str = "ASC25049"  # TACC allocation to charge
app_id_to_use = "mpm-s3"  # MPM application ID

In [4]:
# Convert DesignSafe path to Tapis URI format
input_uri = ds.files.translate_path_to_uri(ds_path)
print(f"Input Directory Tapis URI: {input_uri}")

Translated '/MyData/mpm-benchmarks/2d/uniaxial_stress/' to 'tapis://designsafe.storage.default/kks32/mpm-benchmarks/2d/uniaxial_stress/' using t.username
Input Directory Tapis URI: tapis://designsafe.storage.default/kks32/mpm-benchmarks/2d/uniaxial_stress/


In [5]:
# Generate job request dictionary using app defaults
job_dict = ds.jobs.generate_request(
    app_id=app_id_to_use,
    input_dir_uri=input_uri,
    script_filename=input_filename,
    max_minutes=max_job_minutes,
    allocation=tacc_allocation,
)
print(json.dumps(job_dict, indent=2, default=str))

Generating job request for app 'mpm-s3'...
Using App Details: mpm-s3 v1.0
Found exact match for input parameter: 'Input Directory'
Placing script 'mpm.json' in appArgs: 'Input Script'
Adding/Updating TACC allocation: ASC25049
Job request dictionary generated successfully.
{
  "name": "mpm-s3-20250609_214141",
  "appId": "mpm-s3",
  "appVersion": "1.0",
  "description": "Material Point Method (MPM) is a particle based method that represents the material as a collection of material points, and their deformations are determined by Newton\u2019s laws of motion.",
  "execSystemId": "stampede3",
  "archiveSystemId": "stampede3",
  "archiveSystemDir": "HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}",
  "archiveOnAppError": true,
  "execSystemLogicalQueue": "skx-dev",
  "nodeCount": 1,
  "coresPerNode": 48,
  "maxMinutes": 10,
  "memoryMB": 192000,
  "isMpi": false,
  "tags": [],
  "fileInputs": [
    {
      "name": "Input Directory",
      "sourceUrl": "tapis://de

In [6]:
# Customize job settings (optional)
job_dict["nodeCount"] = 1  # Use single node
job_dict["coresPerNode"] = 1  # Use single core
print(json.dumps(job_dict, indent=2, default=str))

{
  "name": "mpm-s3-20250609_214141",
  "appId": "mpm-s3",
  "appVersion": "1.0",
  "description": "Material Point Method (MPM) is a particle based method that represents the material as a collection of material points, and their deformations are determined by Newton\u2019s laws of motion.",
  "execSystemId": "stampede3",
  "archiveSystemId": "stampede3",
  "archiveSystemDir": "HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}",
  "archiveOnAppError": true,
  "execSystemLogicalQueue": "skx-dev",
  "nodeCount": 1,
  "coresPerNode": 1,
  "maxMinutes": 10,
  "memoryMB": 192000,
  "isMpi": false,
  "tags": [],
  "fileInputs": [
    {
      "name": "Input Directory",
      "sourceUrl": "tapis://designsafe.storage.default/kks32/mpm-benchmarks/2d/uniaxial_stress/",
      "autoMountLocal": true,
      "targetPath": "inputDirectory"
    }
  ],
  "parameterSet": {
    "appArgs": [
      {
        "name": "Input Script",
        "arg": "mpm.json"
      }
    ],
    "sched

In [7]:
# Submit the job to TACC
submitted_job = ds.jobs.submit_request(job_dict)
print(f"Job UUID: {submitted_job.uuid}")


--- Submitting Tapis Job Request ---
{
  "name": "mpm-s3-20250609_214141",
  "appId": "mpm-s3",
  "appVersion": "1.0",
  "description": "Material Point Method (MPM) is a particle based method that represents the material as a collection of material points, and their deformations are determined by Newton\u2019s laws of motion.",
  "execSystemId": "stampede3",
  "archiveSystemId": "stampede3",
  "archiveSystemDir": "HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}",
  "archiveOnAppError": true,
  "execSystemLogicalQueue": "skx-dev",
  "nodeCount": 1,
  "coresPerNode": 1,
  "maxMinutes": 10,
  "memoryMB": 192000,
  "isMpi": false,
  "tags": [],
  "fileInputs": [
    {
      "name": "Input Directory",
      "sourceUrl": "tapis://designsafe.storage.default/kks32/mpm-benchmarks/2d/uniaxial_stress/",
      "autoMountLocal": true,
      "targetPath": "inputDirectory"
    }
  ],
  "parameterSet": {
    "appArgs": [
      {
        "name": "Input Script",
        "arg"

In [8]:
# Monitor job execution until completion
final_status = submitted_job.monitor(interval=15)  # Check every 15 seconds
print(f"Job {submitted_job.uuid} finished with status: {final_status}")


Monitoring Job: 96f1a204-b223-40dc-93ef-98e92046dba0-007


Monitoring job:   0%|          | 0/40 [00:00<?, ? checks/s]                       

	Status: RUNNING


Monitoring job (Status: ARCHIVING):   8%|▊         | 3/40 [00:31<06:47, 11.02s/ checks]

	Status: ARCHIVING


Monitoring job (Status: ARCHIVING): 100%|██████████| 40/40 [01:02<00:00,  1.55s/ checks]

	Status: FINISHED
Job 96f1a204-b223-40dc-93ef-98e92046dba0-007 finished with status: FINISHED


In [11]:
# Interpret and display job outcome
ds.jobs.interpret_status(final_status, submitted_job.uuid)

Job 66778a7f-ac75-486d-91ee-acf82ffe8abe-007 completed successfully.


In [12]:
# Display job runtime summary
submitted_job.print_runtime_summary(verbose=False)


Runtime Summary
---------------
QUEUED  time: 00:00:01
RUNNING time: 00:00:24
TOTAL   time: 00:02:19
---------------


In [13]:
# Get current job status
current_status = ds.jobs.get_status(submitted_job.uuid)
print(f"Current status: {current_status}")

Current status: FINISHED


In [14]:
# Display last status message from TACC
print(f"Last message: {submitted_job.last_message}")

Last message: Setting job status to FINISHED.


In [15]:
# Display job output from stdout
stdout_content = submitted_job.get_output_content("tapisjob.out", max_lines=50)
if stdout_content:
    print("Job output:")
    print(stdout_content)

Attempting to fetch content of 'tapisjob.out' from job archive...
Returning last 50 lines of 'tapisjob.out'.
Job output:
TACC:  Starting parallel tasks... 
[2025-06-05 14:14:37.135] [main] [info] git revision: 18f353fee2ac6735e4f53c9498e7976746b04055
[2025-06-05 14:14:37.160] [MPMBase] [warning] /work2/05873/kks32/stampede3/mpm/include/solvers/mpm_base.tcc #71: Velocity update parameter is not specified, using default as false
[2025-06-05 14:14:37.160] [MPMBase] [warning] /work2/05873/kks32/stampede3/mpm/include/solvers/mpm_base.tcc #94: No math functions are defined
[2025-06-05 14:14:37.160] [MPMBase] [warning] /work2/05873/kks32/stampede3/mpm/include/solvers/mpm_base.tcc #135: No VTK variables were specified, none will be generated
[2025-06-05 14:14:37.160] [MPMBase] [warning] /work2/05873/kks32/stampede3/mpm/include/solvers/mpm_base.tcc #166: No VTK statevariable were specified, none will be generated
[2025-06-05 14:14:37.160] [MPMExplicit] [info] MPM analysis type MPMExplicit2D
[20

In [16]:
# List contents of job archive directory
archive_uri = submitted_job.archive_uri
print(f"Archive URI: {archive_uri}")
outputs = ds.files.list(archive_uri)
for item in outputs:
    print(f"- {item.name} ({item.type})")

Archive URI: tapis://stampede3/work2/05873/kks32/stampede3/tapis-jobs-archive/2025-06-05Z/mpm-s3-20250605_141247-66778a7f-ac75-486d-91ee-acf82ffe8abe-007
Listing files in system 'stampede3' at path 'work2/05873/kks32/stampede3/tapis-jobs-archive/2025-06-05Z/mpm-s3-20250605_141247-66778a7f-ac75-486d-91ee-acf82ffe8abe-007'...
Found 5 items.
- inputDirectory (dir)
- tapisjob.env (file)
- tapisjob.out (file)
- tapisjob.sh (file)
- tapisjob_app.sh (file)
